In [ ]:
Tatenda

In [35]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

In [36]:
metadata=pd.read_csv("movie_metadata_with_score_metacritic.csv", index_col="Unnamed: 0") #reading in metacritic dataset
metadata=metadata.loc[~metadata["metacritic_metascore"].isna()] #removing rows with no metacritic data
metadata=metadata.drop(["homepage","keywords","overview","status","tagline","imdb_metascore","budget","revenue"],1) #dropping unnecessary features
metadata=metadata.drop_duplicates() #dropping duplicates
metadata["release_year"]=metadata.release_date.str[0:4].astype(int) #creating new column with year for comparison

In [37]:
num_data=pd.read_csv("the_numbers_budget.csv") #reading in revenue data
num_data=num_data.rename(columns={"Movie":"title"}) #renaming column to match metadata
num_data["num_year"]=num_data["Release Date"].str[-4:].astype(int)
num_data=num_data.drop_duplicates()#dropping duplicates


In [38]:
critic_revenue=metadata.merge(num_data,on="title") #merging datasets
critic_revenue=critic_revenue.drop_duplicates() #dropping duplicates
critic_revenue=critic_revenue.loc[(critic_revenue["Worldwide Gross"]!=0)] #removing rows with no revenue data
critic_revenue=critic_revenue.loc[(np.abs(critic_revenue.release_year-critic_revenue.num_year)<5)] #removing rows where the years don't match, as this indicates different movies

critic_revenue["log Worldwide Gross"]=np.log(critic_revenue["Worldwide Gross"])#adding column for log of revenue

In [39]:
#Normalizing by the median for merged data set

critic_revenue["Production Budget"]=(critic_revenue["Production Budget"]-critic_revenue["Production Budget"].median())/critic_revenue["Production Budget"].std()
critic_revenue["Worldwide Gross"]=(critic_revenue["Worldwide Gross"]-critic_revenue["Worldwide Gross"].median())/critic_revenue["Worldwide Gross"].std()
critic_revenue["log Worldwide Gross"]=(critic_revenue["log Worldwide Gross"]-critic_revenue["log Worldwide Gross"].median())/critic_revenue["log Worldwide Gross"].std()
critic_revenue["Domestic Gross"]=(critic_revenue["Domestic Gross"]-critic_revenue["Domestic Gross"].median())/critic_revenue["Domestic Gross"].std()
critic_revenue["popularity"]=(critic_revenue["popularity"]-critic_revenue["popularity"].median())/critic_revenue["popularity"].std()
critic_revenue["vote_count"]=(critic_revenue["vote_count"]-critic_revenue["vote_count"].median())/critic_revenue["vote_count"].std()
critic_revenue["metacritic_metascore"]=(critic_revenue["metacritic_metascore"]-critic_revenue["metacritic_metascore"].median())/critic_revenue["metacritic_metascore"].std()
critic_revenue["runtime"]=(critic_revenue["runtime"]-critic_revenue["runtime"].median())/critic_revenue["runtime"].std()

# Reindex since the dataset has been subset many times
critic_revenue.index=range(len(critic_revenue))

In [40]:
#Replace every nan values with 0
critic_revenue.fillna(value=0,axis=1,inplace=True)
critic_revenue.shape

(3319, 21)

In [41]:
#Defining features and target for this dataset based on co-relation
features = ['metacritic_metascore']
target = ['Worldwide Gross']

In [42]:
#splitting data set into training and test data set in 0.7/0.3
train, test = train_test_split(critic_revenue,test_size=0.30)
train.head()

,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,...,vote_average,vote_count,metacritic_metascore,release_year,Release Date,Production Budget,Domestic Gross,Worldwide Gross,num_year,log Worldwide Gross
2770,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",378200,en,The Perfect Match,-0.400200,"[{""name"": ""Lionsgate"", ""id"": 1632}, {""name"": ""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-03-11,-0.439008,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,5.2,-0.234094,-0.765790,2016,3/11/2016,-0.379488,-0.259943,-0.185281,2016,-0.636367
208,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",80274,en,Ender's Game,0.879321,"[{""name"": ""Summit Entertainment"", ""id"": 491}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-23,0.439008,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,6.6,1.467792,-0.218797,2013,11/1/2013,2.110903,0.507815,0.479793,2013,0.481812
650,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",8838,en,Mercury Rising,0.057314,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1998-04-03,0.292672,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,6.0,0.011291,-0.984588,1998,4/3/1998,0.925002,0.083828,-0.056551,1998,-0.117150
2251,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",10220,en,Rounders,0.043474,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1998-09-11,0.780459,"[{""iso_639_1"": ""ru"", ""name"": ""P\u0443\u0441\u0...",...,6.9,0.064733,-0.054699,1998,9/11/1998,-0.213462,-0.064531,-0.113356,1998,-0.277905
979,"[{""id"": 35, ""name"": ""Comedy""}]",16538,en,"I Love You, Man",0.255117,"[{""name"": ""The Montecito Picture Company"", ""id...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-03-20,0.000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,6.5,0.121940,0.820490,2009,3/20/2009,0.450642,0.659335,0.278349,2009,0.337437


In [43]:
#Fill the training and test data with require information
X_train = train[features].dropna()
y_train = train[target].dropna()
X_test = test[features].dropna()
y_test = test[target].dropna()

In [44]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, y_train)

/Users/tatenda/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [45]:
print('Gradient Boosting R squared": %.4f' % model.score(X_test, y_test))

Gradient Boosting R squared": -0.0242


In [46]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
model_mse = mean_squared_error(y_pred, y_test)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)

Gradient Boosting RMSE: 1.0098
